In [1]:
import os
import imageio
import numpy as np

# Crop it (for images from thispersondoesnotexist)
def cropit(image):
    # Crop the image from thispersondoesnotexist here to remove watermarking
    # Originally 1024 x 1024
    orgsize = 1024
    y=55  # that is enough
    height=orgsize-2*y
    x=y
    width=orgsize-2*y    
    image = image[y:y+height, x:x+width]
    
    return image
    
# Gray scale it
def grayscaleit(image):
    # Convert image to grayscale
    image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140])
    
    return image

# Normalize luminance and contrast of grayscale image
def normit(image):
    # Calculate the histogram
    histogram, bins = np.histogram(image.flatten(), bins=256, range=[0, 256])

    # Calculate the cumulative distribution function (CDF)
    cdf = histogram.cumsum()
    cdf_normalized = cdf * histogram.max() / cdf.max()

    # Perform histogram equalization
    equalized_image = np.interp(image.flatten(), bins[:-1], cdf_normalized).reshape(image.shape)

    # Normalize the image to the range [0, 255]
    normalized_image = ((equalized_image - equalized_image.min()) / (equalized_image.max() - equalized_image.min())) * 255

    # Convert the image to 8-bit unsigned integer
    normalized_image = normalized_image.astype(np.uint8)

    return normalized_image

def adjustnormit(image, target_luminance, target_contrast):
    # Calculate the average luminance and average contrast
    average_luminance = np.mean(image)
    average_contrast = np.std(image)

    # Calculate the difference in average values
    delta_luminance = target_luminance - average_luminance
    delta_contrast = target_contrast - average_contrast

    # Adjust the image pixel values based on the differences
    normalized_image = (image + delta_luminance) * (target_contrast / average_contrast)

    # Clip the pixel values to ensure they are within the valid range [0, 255]
    normalized_image = np.clip(normalized_image, 0, 255)

    # Convert the image to 8-bit unsigned integer
    normalized_image = normalized_image.astype(np.uint8)
    
    return normalized_image

# Randomise image phases 
def scramblinspacefreq(image):
    # Convert the image to the frequency domain
    fft_pic = np.fft.fftn(pic, axes=(0, 1))

    # Randomize the phases in the frequency spectrum
    random_phases = np.exp(1j * 2 * np.pi * np.random.rand(*fft_pic.shape))

    # Apply the randomized phases to the original spectrum
    scrambled_fft_pic = np.abs(fft_pic) * random_phases

    # Transform the image back to the spatial domain
    scrambled_pic = np.fft.ifftn(scrambled_fft_pic, axes=(0, 1)).real

    # Normalize the scrambled image to the original image's range
    scrambled_pic = (scrambled_pic - scrambled_pic.min()) / (scrambled_pic.max() - scrambled_pic.min())
    
    return scrambled_pic



In [27]:
## This loop will read images in folder
## Crop 55 pixels on every borders
## Transform to grayscale
## and Save a new folder

# Folder path containing the images
folder_path = r'D:\project_meg-eeg-fMRI\tasks\faces\stim\thispersondoesnotexist'
    
# List all files in the folder
file_list = os.listdir(folder_path)

# Filter only image files (modify the extensions as required)
image_files = [f for f in file_list if f.endswith(('.png', '.jpg', '.gif'))]

# Loop through each image file and apply the transformation
for image_file in image_files:

    # Load the image
    pic = imageio.imread(folder_path+ "\\" +image_file)

    # Crop the image from thispersondoesnotexist here to remove watermarking
    pic = cropit(pic)
    
    # Convert image to grayscale
    pic = grayscaleit(pic)
    
#    # Normalize luminance and contrast
#    pic = normit(pic)
    
    # Normalize to adjusted luminance and contrast 
    target_luminance = 130
    target_contrast = 50
    pic = adjustnormit(pic, target_luminance, target_contrast)
    
    # Save the gray image
    imageio.imwrite(folder_path+"\\gray\\"+"g_"+image_file, np.uint8(pic))

C:\Users\vrochas\AppData\Local\Temp\ipykernel_12292\3824691120.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic = imageio.imread(folder_path+ "\\" +image_file)


In [28]:
## This loop will read images in folder
## randomsied the phases of the image decomposition
## and Save the results

# Folder path containing the images
#folder_path = r'D:\project_meg-eeg-fMRI\tasks\faces\stim\Stirling\frontal_neutral'
folder_path = r'D:\project_meg-eeg-fMRI\tasks\faces\stim\thispersondoesnotexist\gray'
    
# List all files in the folder
file_list = os.listdir(folder_path)

# Filter only image files (modify the extensions as required)
image_files = [f for f in file_list if f.endswith(('.png', '.jpg', '.gif'))]

# Loop through each image file and apply the transformation
for image_file in image_files:

    # Load the image
    pic = imageio.imread(folder_path+ "\\" +image_file)

    # Randomize phases 
    scrambled_pic = scramblinspacefreq(pic)
    
    # Save the scrambled image
    imageio.imwrite(folder_path+"\\"+"scr_"+image_file, np.uint8(scrambled_pic * 255))

C:\Users\vrochas\AppData\Local\Temp\ipykernel_12292\728909054.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic = imageio.imread(folder_path+ "\\" +image_file)


In [26]:
# Print the list of image files in:
folder_path = r'D:\project_meg-eeg-fMRI\tasks\faces\stim\thispersondoesnotexist\gray'
    
# List all files in the folder
file_list = os.listdir(folder_path)

# Filter only image files (modify the extensions as required)
image_files = [f for f in file_list if f.endswith(('.png', '.jpg', '.gif'))]
image_files

['g_fa2_01.jpg',
 'g_fa2_02.jpg',
 'g_fa2_03.jpg',
 'g_fa2_04.jpg',
 'g_fa2_05.jpg',
 'g_fa2_06.jpg',
 'g_fa2_07.jpg',
 'g_fa3_01.jpg',
 'g_fa3_02.jpg',
 'g_fa3_03.jpg',
 'g_fa3_04.jpg',
 'g_fa3_05.jpg',
 'g_fa3_06.jpg',
 'g_fa3_07.jpg',
 'g_fa4_01.jpg',
 'g_fa4_02.jpg',
 'g_fa4_03.jpg',
 'g_fa4_04.jpg',
 'g_fa4_05.jpg',
 'g_fa4_06.jpg',
 'g_fa4_07.jpg',
 'g_ma2_01.jpg',
 'g_ma2_02.jpg',
 'g_ma2_03.jpg',
 'g_ma2_04.jpg',
 'g_ma2_05.jpg',
 'g_ma2_06.jpg',
 'g_ma2_07.jpg',
 'g_ma3_01.jpg',
 'g_ma3_02.jpg',
 'g_ma3_03.jpg',
 'g_ma3_04.jpg',
 'g_ma3_05.jpg',
 'g_ma3_06.jpg',
 'g_ma3_07.jpg',
 'g_ma4_01.jpg',
 'g_ma4_02.jpg',
 'g_ma4_03.jpg',
 'g_ma4_04.jpg',
 'g_ma4_05.jpg',
 'g_ma4_06.jpg',
 'g_ma4_07.jpg',
 'scr_g_fa2_01.jpg',
 'scr_g_fa2_02.jpg',
 'scr_g_fa2_03.jpg',
 'scr_g_fa2_04.jpg',
 'scr_g_fa2_05.jpg',
 'scr_g_fa2_06.jpg',
 'scr_g_fa2_07.jpg',
 'scr_g_fa3_01.jpg',
 'scr_g_fa3_02.jpg',
 'scr_g_fa3_03.jpg',
 'scr_g_fa3_04.jpg',
 'scr_g_fa3_05.jpg',
 'scr_g_fa3_06.jpg',
 'scr_g_fa3_0

In [4]:
## Here is an extra function in order to rescale an image from another set (the control images)

import imageio
import scipy.ndimage

def rescale_image(image, new_size):
    
    # Rescale the image using scipy.ndimage.zoom
    scaled_image = scipy.ndimage.zoom(image, (new_size[0] / image.shape[0], new_size[1] / image.shape[1]))

    return scaled_image

In [5]:
## This loop will read images in folder
## Transform to grayscale
## Rescale to desired size
## and Save a new folder

# Folder path containing the images
folder_path = r'D:\project_meg-eeg-fMRI\tasks\faces\stim\control_images(stablediff)'
    
# List all files in the folder
file_list = os.listdir(folder_path)

# Filter only image files (modify the extensions as required)
image_files = [f for f in file_list if f.endswith(('.png', '.jpg', '.gif'))]

# Loop through each image file and apply the transformation
for image_file in image_files:

    # Load the image
    pic = imageio.imread(folder_path+ "\\" +image_file)

    # Convert image to grayscale
    pic = grayscaleit(pic)
    
    # Rescale
    new_size = (914, 914)
    pic = rescale_image(pic, new_size)
    
    # Normalize to adjusted luminance and contrast 
    target_luminance = 130
    target_contrast = 50
    pic = adjustnormit(pic, target_luminance, target_contrast)
    
    # Save the gray image
    imageio.imwrite(folder_path+"\\rescale\\"+"s_"+image_file, np.uint8(pic))

C:\Users\vrochas\AppData\Local\Temp\ipykernel_7836\242208244.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic = imageio.imread(folder_path+ "\\" +image_file)
